In [1]:
# Methylation levels in the enhancers

In [2]:
# parameters
origin_cell = 'H1'


In [3]:
import pandas as pd

# Load Peaks
df_enhancers = pd.read_csv(f'data/{origin_cell}_enhancers_hg38.bed', sep='\t', header=None)

origin_mehyl_plus = f'data/{origin_cell}_wgbs_cpg_plus.bigWig'
origin_mehyl_minus = f'data/{origin_cell}_wgbs_cpg_minus.bigWig'


In [4]:
df_enhancers

,0,1,2,3
0,chr1,694420,694950,15.760154
1,chr1,698510,699160,19.016000
2,chr1,843600,843970,15.500808
3,chr1,845270,846050,8.262514
4,chr1,891300,892490,13.201270
...,...,...,...,...
59618,chrY,19075454,19075934,8.027802
59619,chrY,19567484,19567994,7.705359
59620,chrY,19744554,19744664,8.369712
59621,chrY,20575884,20576274,8.274048


In [5]:
# unique in df_enhancers[0]
df_enhancers[0].unique()

array(['chr1', 'chr4_GL000008v2_random', 'chr1_KI270706v1_random',
       'chr21', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14',
       'chr15', 'chr15_KI270850v1_alt', 'chr16', 'chr22', 'chr17',
       'chr17_KI270862v1_alt', 'chr17_KI270909v1_alt', 'chr18', 'chr19',
       'chr19_KI270938v1_alt', 'chr2', 'chr20', 'chr22_KI270879v1_alt',
       'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chrX',
       'chr7_KI270803v1_alt', 'chr8', 'chrY'], dtype=object)

In [6]:
# Drop rows where chr has underscore

df_enhancers = df_enhancers[~df_enhancers[0].str.contains('_')]


In [7]:
# This is GRH38, get the sequence

hg38_path = '/home/solozabal/Documents/github/build-deepsea-training-dataset/data_hg38/hg38.fa'

In [8]:
from pyfaidx import Fasta

# Get the sequence
hg38 = Fasta(hg38_path)

def get_sequence(row):
    chrom = row[0]
    start = row[1]
    end = row[2]

    sequence = str(hg38[chrom][start:end])
    sequence = sequence.upper()

    return sequence

df_enhancers['sequence'] = df_enhancers.apply(get_sequence, axis=1)

/tmp/ipykernel_2276951/1858832806.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_enhancers['sequence'] = df_enhancers.apply(get_sequence, axis=1)


In [9]:
# What are the CpG methylation levels
 
import pyBigWig

bw_plus = pyBigWig.open(origin_mehyl_plus)
bw_minus = pyBigWig.open(origin_mehyl_minus)

def get_methylation_level(row):
    chrom = row[0]
    start = row[1]
    end = row[2]
    
    # Get the methylation level over the region
    methylation_plus = bw_plus.values(chrom, start, end)
    methylation_minus = bw_minus.values(chrom, start, end)
    
    return methylation_plus, methylation_minus

# Apply the function to get methylation levels
methylation_levels = df_enhancers.apply(get_methylation_level, axis=1)

# Assing
df_enhancers['methyl_plus'] = methylation_levels.apply(lambda x: x[0])
df_enhancers['methyl_minus'] = methylation_levels.apply(lambda x: x[1])

/tmp/ipykernel_2276951/2925138731.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_enhancers['methyl_plus'] = methylation_levels.apply(lambda x: x[0])
/tmp/ipykernel_2276951/2925138731.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_enhancers['methyl_minus'] = methylation_levels.apply(lambda x: x[1])


In [10]:
df_enhancers

,0,1,2,3,sequence,methyl_plus,methyl_minus
0,chr1,694420,694950,15.760154,TATGACTAAAGGTGAATTCCAGAGCAACATTAAATGTTGTCCCTTT...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,chr1,698510,699160,19.016000,TTTCATGGAATCACAGTTCCATGTGGATGGGGAGGCCTCACAATCA...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,chr1,843600,843970,15.500808,CCTACTCACTCGAATCACTAGAGAATACCAAATAGGAATAGGAAGA...,"[0.0, 0.0, nan, nan, 0.0, nan, 8.3333330154418...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,chr1,845270,846050,8.262514,GCAATGGCTCACAAGGTACTAGTGGAACCCCAGTAAGTTATCTCAG...,"[nan, 0.0, nan, nan, nan, nan, nan, 0.0, nan, ...","[0.0, nan, nan, nan, nan, 0.0, 0.0, nan, nan, ..."
4,chr1,891300,892490,13.201270,TATTTATTACTTCATTAAGAGCAAATAAATACTTTAAGAAAACCTT...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
...,...,...,...,...,...,...,...
59618,chrY,19075454,19075934,8.027802,TAAAGCAAAGGGAGCTGTCAGAGAATCTTCATTTTCCCAGTGTCTT...,"[nan, nan, nan, nan, nan, 0.0, nan, nan, nan, ...","[nan, nan, nan, nan, 0.0, nan, nan, nan, nan, ..."
59619,chrY,19567484,19567994,7.705359,GGGTTCCTGTCCAGAGGGGTAGAGAAGAGTGGCTGAGGGCGCGCCC...,"[nan, nan, nan, nan, nan, 0.0, 0.0, nan, nan, ...","[0.0, 0.0, 0.0, nan, nan, nan, nan, nan, 0.0, ..."
59620,chrY,19744554,19744664,8.369712,TGTAAAATAAGAATCACATTGTCTTTAATGACGCGCTGGTTCCTCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, 0.0, nan, nan, nan, nan, nan, nan, nan, ..."
59621,chrY,20575884,20576274,8.274048,AAGGTACTGCTGTAAGCCTCTGGGACTATACCTCGGCTTGCTCTGC...,"[nan, nan, nan, nan, nan, nan, 0.0, nan, nan, ...","[nan, nan, 0.0, 0.0, nan, nan, nan, nan, 0.0, ..."


In [11]:
# Plot the average methylation levels

import matplotlib.pyplot as plt
import numpy as np
# Compute the mean methylation levels per enhancer removing NaNs
df_enhancers['mean_methyl_plus'] = df_enhancers['methyl_plus'].apply(lambda x: np.nanmean(x) if isinstance(x, list) and len(x) > 0 else None)
df_enhancers['mean_methyl_minus'] = df_enhancers['methyl_minus'].apply(lambda x: np.nanmean(x) if isinstance(x, list) and len(x) > 0 else None)


/tmp/ipykernel_2276951/4220811859.py:6: RuntimeWarning: Mean of empty slice
  df_enhancers['mean_methyl_plus'] = df_enhancers['methyl_plus'].apply(lambda x: np.nanmean(x) if isinstance(x, list) and len(x) > 0 else None)
/tmp/ipykernel_2276951/4220811859.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_enhancers['mean_methyl_plus'] = df_enhancers['methyl_plus'].apply(lambda x: np.nanmean(x) if isinstance(x, list) and len(x) > 0 else None)
/tmp/ipykernel_2276951/4220811859.py:7: RuntimeWarning: Mean of empty slice
  df_enhancers['mean_methyl_minus'] = df_enhancers['methyl_minus'].apply(lambda x: np.nanmean(x) if isinstance(x, list) and len(x) > 0 else None)
/tmp/ipykernel_2276951/4220811859.py:7: SettingWithCopyWarning: 
A value is trying to 

In [12]:
df_enhancers

,0,1,2,3,sequence,methyl_plus,methyl_minus,mean_methyl_plus,mean_methyl_minus
0,chr1,694420,694950,15.760154,TATGACTAAAGGTGAATTCCAGAGCAACATTAAATGTTGTCCCTTT...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN
1,chr1,698510,699160,19.016000,TTTCATGGAATCACAGTTCCATGTGGATGGGGAGGCCTCACAATCA...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN
2,chr1,843600,843970,15.500808,CCTACTCACTCGAATCACTAGAGAATACCAAATAGGAATAGGAAGA...,"[0.0, 0.0, nan, nan, 0.0, nan, 8.3333330154418...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",6.499396,5.769231
3,chr1,845270,846050,8.262514,GCAATGGCTCACAAGGTACTAGTGGAACCCCAGTAAGTTATCTCAG...,"[nan, 0.0, nan, nan, nan, nan, nan, 0.0, nan, ...","[0.0, nan, nan, nan, nan, 0.0, 0.0, nan, nan, ...",3.663194,4.869898
4,chr1,891300,892490,13.201270,TATTTATTACTTCATTAAGAGCAAATAAATACTTTAAGAAAACCTT...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",4.368175,4.880085
...,...,...,...,...,...,...,...,...,...
59618,chrY,19075454,19075934,8.027802,TAAAGCAAAGGGAGCTGTCAGAGAATCTTCATTTTCCCAGTGTCTT...,"[nan, nan, nan, nan, nan, 0.0, nan, nan, nan, ...","[nan, nan, nan, nan, 0.0, nan, nan, nan, nan, ...",0.501047,2.491228
59619,chrY,19567484,19567994,7.705359,GGGTTCCTGTCCAGAGGGGTAGAGAAGAGTGGCTGAGGGCGCGCCC...,"[nan, nan, nan, nan, nan, 0.0, 0.0, nan, nan, ...","[0.0, 0.0, 0.0, nan, nan, nan, nan, nan, 0.0, ...",1.969697,0.225225
59620,chrY,19744554,19744664,8.369712,TGTAAAATAAGAATCACATTGTCTTTAATGACGCGCTGGTTCCTCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, 0.0, nan, nan, nan, nan, nan, nan, nan, ...",0.000000,0.653595
59621,chrY,20575884,20576274,8.274048,AAGGTACTGCTGTAAGCCTCTGGGACTATACCTCGGCTTGCTCTGC...,"[nan, nan, nan, nan, nan, nan, 0.0, nan, nan, ...","[nan, nan, 0.0, 0.0, nan, nan, nan, nan, 0.0, ...",1.010101,0.000000


In [13]:

def classify_cpgs(row):

    methyl_count = {'no-methyl': 0, 'cg-methyl': 0, 'c-methyl': 0, 'g-methyl': 0}

    for position in range(0, len(row['sequence'])-1):

        # Get methylation values at positions 3 and 4 (0-indexed)
        val_c = row['methyl_plus'][position]
        val_g = row['methyl_minus'][position+1]

        # Set threasholds
        threshold_down = 10
        threshold_up = 100 - threshold_down

        # Check if nan
        if np.isnan(val_c) or np.isnan(val_g):
            continue


        # Check for no methylation (both values are below the lower threshold)
        if val_c < threshold_down and val_g < threshold_down:
            methyl_count['no-methyl'] += 1
        
        # Check for cg methylation (both values are above the upper threshold)
        elif val_c > threshold_up and val_g > threshold_up:
            methyl_count['cg-methyl'] += 1
        
        # Check for only c methyl: C is high and G is low
        elif val_c > threshold_up and val_g < threshold_down:
            methyl_count['c-methyl'] += 1
        
        # Check for only g methyl: G is high and C is low
        elif val_g > threshold_up and val_c < threshold_down:
            methyl_count['g-methyl'] += 1
    
    return methyl_count

# df_enhancers['methylation_class'] = df_enhancers.apply(classify_peak, axis=1)

# Show tqdm progress bar
from tqdm import tqdm
tqdm.pandas()

df_enhancers['methylation_class'] = df_enhancers.progress_apply(classify_cpgs, axis=1)

100%|██████████| 59597/59597 [04:29<00:00, 221.22it/s]
/tmp/ipykernel_2276951/789796408.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_enhancers['methylation_class'] = df_enhancers.progress_apply(classify_cpgs, axis=1)


In [14]:
df_enhancers

,0,1,2,3,sequence,methyl_plus,methyl_minus,mean_methyl_plus,mean_methyl_minus,methylation_class
0,chr1,694420,694950,15.760154,TATGACTAAAGGTGAATTCCAGAGCAACATTAAATGTTGTCCCTTT...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,"{'no-methyl': 0, 'cg-methyl': 0, 'c-methyl': 0..."
1,chr1,698510,699160,19.016000,TTTCATGGAATCACAGTTCCATGTGGATGGGGAGGCCTCACAATCA...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,"{'no-methyl': 0, 'cg-methyl': 0, 'c-methyl': 0..."
2,chr1,843600,843970,15.500808,CCTACTCACTCGAATCACTAGAGAATACCAAATAGGAATAGGAAGA...,"[0.0, 0.0, nan, nan, 0.0, nan, 8.3333330154418...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",6.499396,5.769231,"{'no-methyl': 0, 'cg-methyl': 3, 'c-methyl': 0..."
3,chr1,845270,846050,8.262514,GCAATGGCTCACAAGGTACTAGTGGAACCCCAGTAAGTTATCTCAG...,"[nan, 0.0, nan, nan, nan, nan, nan, 0.0, nan, ...","[0.0, nan, nan, nan, nan, 0.0, 0.0, nan, nan, ...",3.663194,4.869898,"{'no-methyl': 0, 'cg-methyl': 2, 'c-methyl': 0..."
4,chr1,891300,892490,13.201270,TATTTATTACTTCATTAAGAGCAAATAAATACTTTAAGAAAACCTT...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",4.368175,4.880085,"{'no-methyl': 2, 'cg-methyl': 1, 'c-methyl': 0..."
...,...,...,...,...,...,...,...,...,...,...
59618,chrY,19075454,19075934,8.027802,TAAAGCAAAGGGAGCTGTCAGAGAATCTTCATTTTCCCAGTGTCTT...,"[nan, nan, nan, nan, nan, 0.0, nan, nan, nan, ...","[nan, nan, nan, nan, 0.0, nan, nan, nan, nan, ...",0.501047,2.491228,"{'no-methyl': 8, 'cg-methyl': 0, 'c-methyl': 0..."
59619,chrY,19567484,19567994,7.705359,GGGTTCCTGTCCAGAGGGGTAGAGAAGAGTGGCTGAGGGCGCGCCC...,"[nan, nan, nan, nan, nan, 0.0, 0.0, nan, nan, ...","[0.0, 0.0, 0.0, nan, nan, nan, nan, nan, 0.0, ...",1.969697,0.225225,"{'no-methyl': 22, 'cg-methyl': 0, 'c-methyl': ..."
59620,chrY,19744554,19744664,8.369712,TGTAAAATAAGAATCACATTGTCTTTAATGACGCGCTGGTTCCTCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, 0.0, nan, nan, nan, nan, nan, nan, nan, ...",0.000000,0.653595,"{'no-methyl': 3, 'cg-methyl': 0, 'c-methyl': 0..."
59621,chrY,20575884,20576274,8.274048,AAGGTACTGCTGTAAGCCTCTGGGACTATACCTCGGCTTGCTCTGC...,"[nan, nan, nan, nan, nan, nan, 0.0, nan, nan, ...","[nan, nan, 0.0, 0.0, nan, nan, nan, nan, 0.0, ...",1.010101,0.000000,"{'no-methyl': 18, 'cg-methyl': 0, 'c-methyl': ..."


In [15]:
# Save the data
df_enhancers.to_csv(f'processed/{origin_cell}_enhancers.csv', sep='\t', index=False)

